### Get all URLs without specifying coordinates

In [1]:
import math
import requests
import numpy as np
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

# Change file path
path = 'C:\\Users\\XXX\\Downloads\\'

# Get all URLs
pre_url = 'http://magnifier.flashphotography.com/MagnifyRender.ashx?X={}&Y={}&O=27226345&'
mid_urls = ['R=00001&F=0223','R=00003&F=0742','R=00003&F=0743','R=00003&F=0744']
post_url = '&A=71714'

for R in range(6,16):
    for F in range(1031,1033):
        mid_urls.append('R=' + str(R).zfill(5) + '&F=' + str(F))

urls = [pre_url + url + post_url for url in mid_urls]

### Functions: Coordinates capture and image processing

In [2]:
# Function: Center point coordinates capture
def coordinates_capture(overlap_w, overlap_h):
    num_w = math.ceil((480 - overlap_w) / (117 - overlap_w))
    num_h = math.ceil((630 - overlap_h) / (117 - overlap_h))
    xs = [57 + (117-overlap_w)*i for i in range(num_w)]
    ys = [58 + (117-overlap_h)*j for j in range(num_h)]
    return xs, ys

# Fuction: Image cropping
def image_crop(url0, x, y):
    url1 = url0.format(x, y)
    content = requests.get(url1).content
    img = Image.open(BytesIO(content))
    crop_img = img.crop((36,36,153,153)) # Grab 117*117 px
    return crop_img

# Function: Concatenate multiple images
def image_concat(imgs, xs, ys):
    target = Image.new('RGB', (480, 630))
    for i in range(len(xs)):
        for j in range(len(ys)):
            target.paste(imgs[len(ys)*i+j], (xs[i]-58, ys[j]-58))
    return target

# Fuction: Image stitching (up and down)
def image_stitch_UD(img1, img2, overlap, k):
    img1, img2 = np.float32(img1), np.float32(img2)
    # Calculate weights for each image
    w = 1 / (1 + np.exp(-k*np.arange(-overlap/2, overlap/2)))
    weights1 = w[::-1].reshape(-1, 1, 1)  # Weight for img1 decreases
    weights2 = w.reshape(-1, 1, 1)  # Weight for img2 increases
    # Apply weighted smoothing to eliminate seams
    seam_smoothed = img1[-overlap:, :] * weights1 + img2[:overlap, :] * weights2
    # Combine images with only non-overlapping regions from each
    result = np.vstack((img1[:-overlap, :], seam_smoothed, img2[overlap:, :]))
    target = Image.fromarray(np.uint8(result))
    return target

# Fuction: Image stitching (left and right)
def image_stitch_LR(img1, img2, overlap, k):
    img1, img2 = np.float32(img1), np.float32(img2)
    # Calculate weights for each image
    w = 1 / (1 + np.exp(-k*np.arange(-overlap/2, overlap/2)))
    weights1 = w[::-1].reshape(1, -1, 1)  # Weight for img1 decreases
    weights2 = w.reshape(1, -1, 1)  # Weight for img2 increases
    # Apply weighted smoothing to eliminate seams
    seam_smoothed = img1[:, -overlap:] * weights1 + img2[:, :overlap] * weights2
    # Combine images with only non-overlapping regions from each
    result = np.hstack((img1[:, :-overlap], seam_smoothed, img2[:, overlap:]))
    target = Image.fromarray(np.uint8(result))
    return target

### Save high-resolution pictures

In [3]:
urls = urls[0:1]

# It is wise to preload web pages with images to be processed to avoid unforseen errors
if __name__ == '__main__':
    overlap_w, overlap_h = 42, 50
    
    # Center point coordinates
    (xs, ys) = coordinates_capture(overlap_w, overlap_h)
    
    # Run functions and save high-resolution pictures
    for i in range(len(urls)):
        imgs = []
        for x in xs:
            for y in ys:
                imgs.append(image_crop(urls[i], x, y))
        
        picture = image_concat(imgs, xs, ys)
        picture = picture.crop((1,3,480,630))
        picture.save(path + 'picture_' + str(i+1) + '.jpg', quality=95)
        # plt.imshow(picture)
        # plt.show()

In [4]:
# It is wise to preload web pages with images to be processed to avoid unforseen errors
if __name__ == '__main__':
    overlap_w, overlap_h = 42, 50
    
    # Adjust k to obtain the best image fusion effect
    k = 0.2
    
    # Center point coordinates
    (xs, ys) = coordinates_capture(overlap_w, overlap_h)
    
    # Run functions and save high-resolution pictures
    for i in range(len(urls)):
        himgs = []
        for x in xs:
            vimgs = []
            for y in ys:
                vimgs.append(image_crop(urls[i], x, y))
            vimg = image_stitch_UD(vimgs[0], vimgs[1], overlap_h, k)
            for vi in range(2, len(ys)):
                vimg = image_stitch_UD(vimg, vimgs[vi], overlap_h, k)
            himgs.append(vimg)
        himg = image_stitch_LR(himgs[0], himgs[1], overlap_w, k)
        for hi in range(2, len(xs)):
            himg = image_stitch_LR(himg, himgs[hi], overlap_w, k)
        target_pic = himg.crop((2,2,482,632))
        target_pic.save(path + 'picture_' + str(i+1) + '.jpg', quality=95)
        # plt.imshow(target_pic)
        # plt.show()